In [ ]:
from fisherA2Z.fisher import Fisher

import pyccl as ccl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cosmo = ccl.Cosmology(Omega_c=0.2666, 
                       Omega_b=0.049, 
                       h=0.6727, 
                       sigma8=0.831, 
                       n_s=0.9645, 
                       transfer_function='eisenstein_hu')


In [ ]:
fisher_obj = Fisher(cosmo, end = 7)

In [ ]:
fisher_obj.process()

In [ ]:
fisher_obj_y1 = Fisher(cosmo, end = 7, y1 = True)

In [ ]:
fisher_obj_y1.process()

In [ ]:
from fisherA2Z.fisher import FoM
from fisherA2Z.fisher import Fisher, marginalize, SmailZ, Core, PhotozModel
import numpy as np
import matplotlib

In [ ]:
def plot_contours(matrix, sigmas, fid,  **kwargs):
    prefactor = {1:1.52, 2:2.48}
    prefactor = prefactor[sigmas]
    matrix = np.linalg.inv(matrix)
    s00, s01, s11 = matrix[0][0], matrix[0][1], matrix[1][1]
    a = np.sqrt(
        0.5*(s00 + s11) + np.sqrt(s01**2 + 0.25*(s00-s11)**2)
    )
    b = np.sqrt(
        0.5*(s00 + s11) - np.sqrt(s01**2 + 0.25*(s00-s11)**2)
    )
    b *= prefactor
    a *= prefactor
    theta = np.arctan(2*s01/(s00-s11))/2
    eig = np.linalg.eig(matrix)
    maxeig = eig[1][np.argmax(eig[0])]
    theta = np.arctan2(maxeig[1], maxeig[0])
    el = matplotlib.patches.Ellipse(fid, 2*a, 2*b, angle=-np.degrees(theta), alpha=0.3, **kwargs)
    xlim = np.sqrt(a**2*np.cos(theta)**2 + b**2*np.sin(theta)**2)
    ylim = np.sqrt(a**2*np.sin(theta)**2 + b**2*np.cos(theta)**2)
    return el, xlim, ylim, np.pi*a*b

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16,8))

for paramset in [[['omega_m', 'sigma_8'], ['w_0', 'w_a']]]:
    for j, params in enumerate(paramset):
        ax[j].set_title('Cosmic Shear Only', fontsize = 22)
        colors = ['red', 'green']
        arg = fisher_obj
        labels = ['3x2pt Y10', '3x2pt Y1']
        es = []
        for matrix, label, color in zip([fisher_obj_y1.fisher, fisher_obj.fisher], labels, colors):
            fid=(arg.vals[params[0]], arg.vals[params[1]])
            if label == 'biased':
                fid = (fid[0] + para_bias[params[0]], fid[1] + para_bias[params[1]])
                
            this_fisher = marginalize(matrix, 
                            arg.param_order.index(params[0]), 
                            arg.param_order.index(params[1]))
            e, xlim, ylim, _ = plot_contours(
                this_fisher,
                sigmas=2, fid=fid
                )
            
                    
            e.set_label(label)
            e.set_facecolor('none')
            e.set_alpha(1)
            e.set_edgecolor(color)
            e.set_linewidth(3.0)
            es.append(e)
            ax[j].add_artist(e)
            #limnew = 1*max(abs(para_bias[params[0]]), abs(para_bias[params[1]]))
            xlim = (arg.vals[params[0]] - 4* xlim, arg.vals[params[0]] + 4* xlim)
            ylim = (arg.vals[params[1]] - 4* ylim, arg.vals[params[1]] + 4* ylim)
            ax[j].set_xlim((xlim))
            ax[j].set_ylim((ylim))
            
            ax[j].set_xlabel(arg.param_labels[arg.param_order.index(params[0])],fontsize = 22)
            ax[j].set_ylabel(arg.param_labels[arg.param_order.index(params[1])],fontsize = 22)
            plt.tight_layout()
    
    ax[j].legend(handles=es)
    
        